In [1]:
import datetime as datetime
import datetime 
import pandas as pd
import numpy as np

In [2]:
devices = {}
devices[0] = pd.read_csv("devices_featured0.csv")
devices[1] = pd.read_csv("devices_featured1.csv")
devices[2] = pd.read_csv("devices_featured2.csv")
devices[3] = pd.read_csv("devices_featured3.csv")
#devices[6] = pd.read_csv("devices_featured6.csv")
auctions = pd.read_csv("auctions.csv.gzip", compression='gzip')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### Transformo a datetime la fecha y las posiciones de clicks en floats

In [3]:
auctions['date'] = pd.to_datetime(auctions['date'])

### Agrego columnas a auctions para poder clasificar despues

In [4]:
auctions['auction_day'] = auctions['date'].dt.day

# Appendeando el dataframe

### Separo los dataframe para obtener las distintas ventanas con las cuales entrenar

In [5]:
df_auctions = {}
for i in range(18, 22):
#for i in range (24, 25):
    windowStartDate = datetime.datetime(2019, 4, i)
    windowEndDate = datetime.datetime(2019, 4, i + 3)
    df_auctions[i - 18] = auctions.loc[(auctions['date'] >= windowStartDate) & (windowEndDate >= auctions['date'])]
    #Calculando diferencias de tiempos para auctions para un mismo usuario y esa ventana
    df_auctions[i - 18]['diff_to_prev_auction'] = df_auctions[i - 18].sort_values('date', ascending=True).groupby('device_id')['date'].diff()
    df_auctions[i - 18]['diff_to_prev_auction'] = df_auctions[i - 18]['diff_to_prev_auction']/np.timedelta64(1, 's')
    
    #Agrego los segundos a la proxima subasta (desde el inicio de la ventana)
    next_window_auctions = auctions.loc[(auctions['date'] >= datetime.datetime(2019, 4, i + 3)) & (datetime.datetime(2019, 4, i + 6) >= auctions['date'])]
    devices[i - 18] = pd.merge(devices[i - 18], next_window_auctions.groupby('device_id')['date'].min().reset_index(), left_on='ref_hash', right_on='device_id', how='left')
    devices[i - 18].rename(columns = {'date':'seconds_to_auction'}, inplace=True)
    devices[i - 18]['seconds_to_auction'] = (devices[i - 18]['seconds_to_auction'] - windowEndDate)/np.timedelta64(1, 's')
    devices[i - 18].drop('device_id', inplace=True, axis=1)
    
    #Completo los seconds_to_auction de los que no tuvieron subastas con el valor correspondiente a 3 dias en segundos
    devices[i - 18].loc[pd.isnull(devices[i - 18]['seconds_to_auction']), 'seconds_to_auction'] = 259200

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Agrego features para cada dataframe de devices

In [6]:
for i in range(0, 4):
#for i in range (6, 7):
    #Porcentaje de subastas por dia en el periodo + total subastas
    auctions_by_day = pd.pivot_table(df_auctions[i], index='device_id', columns='auction_day', values='date', aggfunc='count', fill_value=0, margins=True).reset_index()
    auctions_by_day.drop(auctions_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    auctions_by_day.columns = ['ref_hash', 'auctions_day_1', 'auctions_day_2', 'auctions_day_3', 'total_auctions']
    auctions_by_day['ref_hash'] = auctions_by_day['ref_hash'].astype('int64')
    auctions_by_day['auctions_day_1_percent'] = auctions_by_day['auctions_day_1']/auctions_by_day['total_auctions']
    auctions_by_day['auctions_day_2_percent'] = auctions_by_day['auctions_day_2']/auctions_by_day['total_auctions']
    auctions_by_day['auctions_day_3_percent'] = auctions_by_day['auctions_day_3']/auctions_by_day['total_auctions']
    devices[i] = pd.merge(devices[i], auctions_by_day, on='ref_hash', how='left')
    devices[i]['total_auctions_normalized'] = devices[i]['total_auctions']/(devices[i]['total_auctions'].max())
    del auctions_by_day
    
    auctions_grouped = df_auctions[i].groupby('device_id').agg({'diff_to_prev_auction': ['mean', 'std']}).reset_index()
    auctions_grouped.columns = ['ref_hash', 'mean_diff_between_auctions', 'std_diff_between_auctions']
    devices[i] = pd.merge(devices[i], auctions_grouped, on='ref_hash', how='left')
    #Completo el mean_diff_between_auction con 259200 (3 dias en segundos) y el std_diff_between_auction con 0 (el desvio estandar de una muestra unica es 0)
    devices[i]['mean_diff_between_auctions'].fillna(259200, inplace=True)
    devices[i]['std_diff_between_auctions'].fillna(0, inplace=True)
    del auctions_grouped
    
    #Completo los nan con 0 todas las columnas restantes
    devices[i].fillna(0, inplace=True)

In [7]:
#Libero memoria
import gc
gc.collect()

407

In [8]:
for i in range(0, 4):
#for i in range(6, 7):
    devices[i].to_csv('devices_featured{}_final.csv'.format(i), header=True, index=None)
    print('Generado el archivo devices_featured{}_final.csv'.format(i))

Generado el archivo devices_featured0_final.csv
Generado el archivo devices_featured1_final.csv
Generado el archivo devices_featured2_final.csv
Generado el archivo devices_featured3_final.csv
